In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
file_path = '/content/drive/MyDrive/hw/causal/chicago-police/' # change this

# Load the datasets
complaints_allegation = pd.read_csv(file_path + 'complaints-allegation.csv')
officers = pd.read_csv(file_path + 'final-profiles.csv')
salary = pd.read_csv(file_path + 'salary_2002-2017_2017-09.csv')
salary_ranks = pd.read_csv(file_path + 'salary-ranks_2002-2017_2017-09.csv')
merged_data_1 = pd.merge(salary_ranks, complaints_allegation, left_on='link_UID', right_on='link_UID', how='left')
merged_data = pd.merge(merged_data_1, officers, left_on='link_UID', right_on='link_UID', how='left')
# selected_columns = merged_data[['link_UID', 'race','gender','birth_year','salary', 'appointed_date', 'cleaned_rank_x','allegation_category','final_finding','penalty_code','number_of_days']]
# selected_columns.sample(n=10)
sustained_df = merged_data[merged_data['final_finding'] == 'SUSTAINED']
outcome_counts = sustained_df['allegation_category'].value_counts(dropna=False)
# outcome_counts.head(10)
category_mapping = {
    'MISCELLANEOUS': 'Non-Violent',
    'DOMESTIC ALTERCATION - PHYSICAL ABUSE': 'Violent',
    'NEGLECT OF DUTY': 'Non-Violent',
    'EXCESSIVE FORCE - USE OF FIREARM / OFF DUTY - INJURY': 'Violent',
    'UNNECESSARY DISPLAY OF WEAPON / ON DUTY': 'Non-Violent',
    'USE OF PROFANITY': 'Non-Violent',
    'EXCESSIVE FORCE / OFF DUTY - NO INJURY': 'Violent',
    'EXCESSIVE FORCE / On DUTY - INJURY': 'Violent',
    'CONDUCT UNBECOMING': 'Non-Violent',
    'NO INJURY': "Non-Violent",
    'PUSH/PULL/GRAB': "Violent",
    'INJURY': "Violent",
    'FALSE': "Non-Violent",
    'FAIL TO SUBMIT': "Non-Violent",
    'INADEQUATE / FAILURE TO PROVIDE SERVICE': "Non-Violent",
    'HARASSMENT': "Non-Violent",
    'EXCESSIVE FORCE - USE OF FIREARM / OFF DUTY - NO INJURY': "Violent",
    'VERBAL ABUSE': "Non-Violent",
    'FAIL TO OBTAIN A COMPLAINT REGISTER NUMBER': "Non-Violent",
    'LEAVING ASSIGNMENT (DISTRICT, BEAT, SECTOR, COURT)': "Non-Violent",
    'INTOXICATED OFF DUTY': "Non-Violent",
    'CLOSED HAND STRIKE (PUNCH)': "Violent",
}
# sustained_df['violence_classification'] = sustained_df['allegation_category'].map(category_mapping).fillna('Non-Violent')
sustained_df.loc[:, 'violence_classification'] = sustained_df['allegation_category'].map(category_mapping).fillna('Non-Violent')


# Combine low-frequency ranks into a single category
low_frequency_ranks = ['CAPTAIN', 'CIVILIAN', 'COMMANDER', 'DEPUTY CHIEF', 'LIEUTENANT', 'INVESTIGATOR']
# sustained_df['combined_rank'] = sustained_df['cleaned_rank_x'].replace(low_frequency_ranks, 'Higher Ranks')
sustained_df.loc[:, 'combined_rank'] = sustained_df['cleaned_rank_x'].replace(low_frequency_ranks, 'Higher Ranks')



# Number of bootstrap samples
n_bootstrap_samples = 1000

# Convert the dataframe to a numpy array for faster operations
# sustained_array = sustained_df[['combined_rank', 'violence_classification']].to_numpy()

# # Unique ranks and their indices
# unique_ranks, rank_indices = np.unique(sustained_array[:, 0], return_inverse=True)

# Function to calculate the proportion of violent allegations by combined rank
# def calculate_proportion(array, rank_indices):
#     proportions = []
#     for rank in np.unique(rank_indices):
#         mask = rank_indices == rank
#         proportion = np.mean(array[mask, 1] == 'Violent')
#         proportions.append(proportion)
#     return np.array(proportions)

# # Initialize list to store bootstrap results
# bootstrap_results = []

# # Bootstrap sampling
# for _ in range(n_bootstrap_samples):
#     bootstrap_sample_indices = np.random.choice(np.arange(sustained_array.shape[0]), size=sustained_array.shape[0], replace=True)
#     bootstrap_sample = sustained_array[bootstrap_sample_indices]
#     bootstrap_results.append(calculate_proportion(bootstrap_sample, rank_indices[bootstrap_sample_indices]))

# # Convert results to DataFrame
# bootstrap_df = pd.DataFrame(bootstrap_results, columns=unique_ranks)

# # Calculate confidence intervals (e.g., 95% CI)
# confidence_intervals = bootstrap_df.quantile([0.025, 0.975])

# # Display the results
# print("Bootstrap Confidence Intervals:")
# print(confidence_intervals)


################################
def calculate_proportion(df):
    return df.groupby('combined_rank')['violence_classification'].apply(lambda x: (x == 'Violent').mean())

# Initialize list to store bootstrap results
bootstrap_results = []

# Bootstrap sampling
for _ in range(n_bootstrap_samples):
    bootstrap_sample = sustained_df.sample(frac=1, replace=True)
    bootstrap_results.append(calculate_proportion(bootstrap_sample))

# Convert results to DataFrame
bootstrap_df = pd.DataFrame(bootstrap_results)

# Calculate confidence intervals (e.g., 95% CI)
confidence_intervals = bootstrap_df.quantile([0.025, 0.975])

# Display the results
print("Bootstrap Confidence Intervals:")
print(confidence_intervals)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-11-766c6e17902c>:8: DtypeWarning: Columns (16,18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_allegation = pd.read_csv(file_path + 'complaints-allegation.csv')
<ipython-input-11-766c6e17902c>:9: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  officers = pd.read_csv(file_path + 'final-profiles.csv')
<ipython-input-11-766c6e17902c>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sustained_df.loc[:, 'violence_classification'] = sustained_df['allegation_category'].map(category_mapping).fillna('Non-Violent')
<ipython-input-11-766c6e17902c>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

Bootstrap Confidence Intervals:
combined_rank  DETECTIVE  FIELD TRAINING OFFICER  Higher Ranks  \
0.025           0.339177                0.288981      0.214309   
0.975           0.355240                0.322605      0.246924   

combined_rank  POLICE OFFICER  SERGEANT  
0.025                0.240160  0.256493  
0.975                0.244084  0.267745  


In [ ]:
print(sustained_df['combined_rank'].value_counts())
print(sustained_df.groupby('combined_rank')['violence_classification'].value_counts())

combined_rank
POLICE OFFICER            206837
SERGEANT                   22464
DETECTIVE                  13559
FIELD TRAINING OFFICER      3071
Higher Ranks                2655
Name: count, dtype: int64
combined_rank           violence_classification
DETECTIVE               Non-Violent                  8853
                        Violent                      4706
FIELD TRAINING OFFICER  Non-Violent                  2133
                        Violent                       938
Higher Ranks            Non-Violent                  2042
                        Violent                       613
POLICE OFFICER          Non-Violent                156756
                        Violent                     50081
SERGEANT                Non-Violent                 16576
                        Violent                      5888
Name: count, dtype: int64


#the rest is copied from synthetic data notebook

In [ ]:
import argparse
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
import random


def get_smf_model_a_param(ols, df):
    """
    Fit a model with statsmodels
    Return the parameter corresponding to the treatment
    """
    return smf.ols(ols, data=df).fit().params['a']


def get_sklearn_model_a_param(ols, df):
    """
    Fit a model with sklearn
    Return the parameter corresponding to the treatment
    """
    target = ols.split("~")[0].strip()
    inputs = ols.split("~")[1].strip().split(" + ")

    model = LinearRegression()
    model.fit(df[inputs], df[target])

    return model.coef_[inputs.index("a")]

def observed(n=100, c_dim=6, ols="y ~ a"):
      # confounders = race (c1) and gender (c2)
    c1 = np.random.binomial(n=1, p=0.423, size=n)

    # choose p based on distribution of male vs female officers in the data
    c2 = np.random.binomial(n=1, p=0.8763, size=n)

    # a = rank/treatment
    a = np.random.randint(1, 7+c1+c2, n)
    # a = np.random.randint(n=1 + c_dim - c, p=0.5, size=n)
    # a = (a > 0).astype(np.int32)
    y = np.random.binomial(n=a + c1+c2, p=0.5)

    df = pd.DataFrame(data=dict(c1=c1, c2=c2, a=a, y=y))
    a_param = get_smf_model_a_param(ols, df)
    # a_param = get_sklearn_model_a_param(ols, df)

    print(a_param)

    return a_param


# def observed(n=100, c_dim=6, ols="y ~ a"):
#     """
#     The observed data distribution
#       C: roll a k-sided die and record the result
#       A: flip `1 + k - C` fair coins, and record 1 if at least one flip lands heads
#       Y: flip `C + A` fair coins, and record the number of heads
#     """
#     # confounders = race (c1) and gender (c2)
#     c1 = np.random.randint(1, 6, n)

#     # choose p based on distribution of male vs female officers in the data
#     c2 = np.random.binomial(n=1, p=0.8763, size=n)

#     # a = rank/treatment
#     a = np.random.binomial(n=1+c1+c2, p=0.5, size=n) * (c1 + c2)
#     # a = np.random.randint(n=1 + c_dim - c, p=0.5, size=n)
#     # a = (a > 0).astype(np.int32)
#     y = np.random.binomial(n=a + c, p=0.5)

#     df = pd.DataFrame(data=dict(c=c, a=a, y=y))
#     a_param = get_smf_model_a_param(ols, df)
#     # a_param = get_sklearn_model_a_param(ols, df)

#     return a_param

# def observed(n=100, ols="y ~ a"):
#     """
#     The observed data distribution
#       C: roll a k-sided die and record the result
#       A: flip `1 + k - C` fair coins, and record 1 if at least one flip lands heads
#       Y: flip `C + A` fair coins, and record the number of heads
#     """
#     # confounders = race (c1) and gender (c2)
#     c1 = np.random.binomial(n=1, p=0.423, size=n) #white prob: 0.423

#     # choose p based on distribution of male vs female officers in the data
#     c2 = np.random.binomial(n=1, p=0.8763, size=n)


#     # a = rank/treatment
#     # a = np.random.binomial(n=1+c1+c2, p=0.5, size=n) * (c1 + c2)
#     # a = random.random()* (c1 + c2)
#     # a = np.random.binomial(n=(1 + c1 + c2), p=0.5, size=n)
#     # p = 0.3 + 0.2 * c1 + 0.2 * c2 + 0.3 * (c1 & c2)  # Increased probability when both c1 and c2 are 1

#     # a = np.random.binomial(n=1, p=p, size=n)
#     min_a = 0
#     max_a = 10
#     max_a_modified = max_a + 5 * c1 + 5 * c2

# # Generate A uniformly within the modified range
#     a = np.random.uniform(min_a, max_a_modified)
#     # a = 7 + c1 + c2 +


#     # a = np.random.randint(n=1 + c_dim - c, p=0.5, size=n)
#     # a = (a > 0).astype(np.int32)
#     y = np.random.binomial(n=np.round(a).astype(int) + c1 + c2, p=0.5,size = n)
#     # y = random.random() * (a + c1 + c2)

#     df = pd.DataFrame(data=dict(c1=c1, c2=c2, a=a, y=y))
#     a_param = get_smf_model_a_param(ols, df)
#     # a_param = get_sklearn_model_a_param(ols, df)

#     return a_param


# def observed(n=100, c_dim=6, ols="y ~ a"):
#     """
#     The observed data distribution
#       C: roll a k-sided die and record the result
#       A: flip `1 + k - C` fair coins, and record 1 if at least one flip lands heads
#       Y: flip `C + A` fair coins, and record the number of heads
#     """
#     birth_year = np.random.randint(1950, 2000, n)
#     gender = np.random.binomial(1, 0.5, n)  # 0 for female, 1 for male
#     race = np.random.randint(0, 3, n)

#     appointed_date = birth_year + np.random.randint(18, 31, n)  # Age at appointment between 18-30
#     rank = gender + (2024 - appointed_date) // 10 + race  # Simplified ranking logic

#     allegation_category = race  # Direct dependency from race

#     # Further dependent variables based on rank
#     final_finding = rank % 3  # Simplified example
#     penalty_code = rank % 5  # Simplified example
#     number_of_days_suspended = final_finding * 2 + penalty_code * 3  # Simplified calculation



#     c = np.random.randint(1, 1 + c_dim, n)
#     a_tmp = np.random.binomial(n=1 + c_dim - c, p=0.5, size=n)
#     a = (a_tmp > 0).astype(np.int32)
#     y = np.random.binomial(n=a + c, p=0.5)

#     df = pd.DataFrame(data=dict(c=c, a=a, y=y))
#     a_param = get_smf_model_a_param(ols, df)

#     return a_param


def randomized(n=100, c_dim = 6, ols="y ~ a"):
    """
    The same distribution, except A is replaced with a fair coin f
      C: roll a k-sided die and record the result
      A: flip a single fair coin, and record 1 if it lands heads
      Y: flip `C + A` fair coins, and record the number of heads
    """

    c = np.random.randint(1, 1 + c_dim, n)
    a = np.random.binomial(n=1, p=0.5, size=n)
    y = np.random.binomial(n=a + c, p=0.5)

    df = pd.DataFrame(data=dict(c=c, a=a, y=y))
    a_param = get_smf_model_a_param(ols, df)

    return a_param
def experiment(dist, n=100, ols="y ~ a", repeats=1):
    """
    Run an experiment with the given kwargs
      dist: either "observed" or "randomized" distribution
      n: the number of samples to draw from the distribution
      c_dim: possible values that C can take (number of sides of the die)
      ols: regression model; either "y ~ a" or "y ~ a + c"
    """
    if dist == "observed":
        func = observed
    elif dist == "randomized":
        func = randomized
    else:
        raise ValueError(dist)

    np.random.seed(42)
    results = [func(n=n, ols=ols) for i in range(repeats)]
    err = ""
    if repeats > 1:
        err = f" ± {np.std(results):.3f}"
    print(f"{np.mean(results):.3f}{err}")



In [ ]:
experiment("observed", n=100, ols="y ~ a", repeats=10)

0.4277590315958376
0.47627478753541075
0.462743032276602
0.5462407862407856
0.5774672915879966
0.4976138828633404
0.5884165946029848
0.635354879451499
0.6311625228473424
0.5244522691705787
0.537 ± 0.068


In [ ]:
experiment("randomized", n=100, ols="y ~ a", repeats=10)

0.586 ± 0.338
